<img src = "https://github.com/VeryFatBoy/notebooks/blob/main/common/images/img_github_singlestore-jupyter_featured_2.png?raw=true">

<div id="singlestore-header" style="display: flex; background-color: rgba(235, 249, 245, 0.25); padding: 5px;">
    <div id="icon-image" style="width: 90px; height: 90px;">
        <img width="100%" height="100%" src="https://raw.githubusercontent.com/singlestore-labs/spaces-notebooks/master/common/images/header-icons/browser.png" />
    </div>
    <div id="text" style="padding: 5px; margin-left: 10px;">
        <div id="badge" style="display: inline-block; background-color: rgba(0, 0, 0, 0.15); border-radius: 4px; padding: 4px 8px; align-items: center; margin-top: 6px; margin-bottom: -2px; font-size: 80%">SingleStore Notebooks</div>
        <h1 style="font-weight: 500; margin: 8px 0 0 4px;">Using SingleStoreDB with ChatGPT to answer questions about the 2022 Winter Olympics</h1>
    </div>
</div>

In [4]:
!pip cache purge --quiet

In [5]:
!pip install openai --quiet
!pip install scikit-learn --quiet
!pip install tiktoken --quiet

In [6]:
import ast
import getpass
import numpy as np
import openai
import os
import pandas as pd
import tiktoken

from IPython.display import display
from openai import OpenAI
from scipy import spatial

In [7]:
# Models
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"

In [8]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
openai.api_key = os.environ["OPENAI_API_KEY"]

client = OpenAI()

OpenAI API Key: ········


## 1. Prepare search data

In [10]:
# Download pre-chunked text and pre-computed embeddings
# This file is ~200 MB, so may take a minute depending on your connection speed
embeddings_path = "https://cdn.openai.com/API/examples/data/winter_olympics_2022.csv"

df = pd.read_csv(embeddings_path)

In [11]:
# Convert embeddings from CSV str type back to list type and convert to float32
df["embedding"] = df["embedding"].apply(
    lambda x: np.array(ast.literal_eval(x), dtype = np.float32)
)

In [13]:
# The dataframe has two columns: "text" and "embedding"
df

,text,embedding
0,Lviv bid for the 2022 Winter Olympics\n\n{{Oly...,"[-0.005021068, 0.00026050032, -0.0046091327, 0..."
1,Lviv bid for the 2022 Winter Olympics\n\n==His...,"[0.003392742, -0.007447326, 0.008918799, 0.005..."
2,Lviv bid for the 2022 Winter Olympics\n\n==Ven...,"[-0.00915789, -0.008366799, -0.004565537, 3.06..."
3,Lviv bid for the 2022 Winter Olympics\n\n==Ven...,"[0.003095189, -0.006064315, 0.013017709, 0.016..."
4,Lviv bid for the 2022 Winter Olympics\n\n==Ven...,"[-0.0029361742, -0.0061851772, 0.005705733, 0...."
...,...,...
6054,Anaïs Chevalier-Bouchet\n\n==Personal life==\n...,"[-0.0277504, 0.0017460181, -0.00618741, 0.0052..."
6055,Uliana Nigmatullina\n\n{{short description|Rus...,"[-0.021714168, 0.016001321, 0.011204938, -0.00..."
6056,Uliana Nigmatullina\n\n==Biathlon results==\n\...,"[-0.029143544, 0.014654332, 0.004843497, -0.00..."
6057,Uliana Nigmatullina\n\n==Biathlon results==\n\...,"[-0.02426604, 0.011665307, 0.009945774, 0.0025..."


## 2. Search

In [14]:
%%sql
CREATE DATABASE IF NOT EXISTS openai_demo;

USE openai_demo;

DROP TABLE IF EXISTS winter_olympics_2022;
CREATE TABLE IF NOT EXISTS winter_olympics_2022 (
    text TEXT CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci,
    embedding VECTOR(1536)
)

1 rows affected.

++
||
++
++

<div class="alert alert-block alert-warning">
    <b class="fa fa-solid fa-exclamation-circle"></b>
    <div>
        <p><b>Action Required</b></p>
        <p>Select the database from the drop-down menu at the top of this notebook. It updates the <b>connection_url</b> which is used by SQLAlchemy to make connections to the selected database.</p>
    </div>
</div>

In [16]:
from sqlalchemy import *

db_connection = create_engine(connection_url)

In [17]:
df.to_sql(
    "winter_olympics_2022",
    con = db_connection,
    if_exists = "append",
    index = False,
    chunksize = 1000
)

6059

In [18]:
def strings_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn = lambda x, y: 1 - spatial.distance.cosine(x, y),
    top_n: int = 100
) -> list:
    """Returns a list of strings and relatednesses, sorted from most related to least."""

    query_embedding_response = client.embeddings.create(
        model = EMBEDDING_MODEL,
        input = query,
    )
    query_embedding = query_embedding_response.data[0].embedding

    results = pd.read_sql("""
        SELECT text, (%s <*> embedding) AS score
        FROM winter_olympics_2022
        ORDER BY score DESC
        LIMIT %s
    """, db_connection, params = (str(query_embedding), top_n))

    # Separate the results into two lists
    strings = results["text"].tolist()
    relatednesses = results["score"].tolist()

    # Return the results
    return strings[:top_n], relatednesses[:top_n]

In [19]:
# Examples
strings, relatednesses = strings_ranked_by_relatedness("curling gold medal", df, top_n=5)
for string, relatedness in zip(strings, relatednesses):
    print(f"{relatedness=:.3f}")
    display(string)

relatedness=0.879


'Curling at the 2022 Winter Olympics\n\n==Medal summary==\n\n===Medal table===\n\n{{Medals table\n | caption        = \n | host           = \n | flag_template  = flagIOC\n | event          = 2022 Winter\n | team           = \n | gold_CAN = 0 | silver_CAN = 0 | bronze_CAN = 1\n | gold_ITA = 1 | silver_ITA = 0 | bronze_ITA = 0\n | gold_NOR = 0 | silver_NOR = 1 | bronze_NOR = 0\n | gold_SWE = 1 | silver_SWE = 0 | bronze_SWE = 2\n | gold_GBR = 1 | silver_GBR = 1 | bronze_GBR = 0\n | gold_JPN = 0 | silver_JPN = 1 | bronze_JPN - 0\n}}'

relatedness=0.872


"Curling at the 2022 Winter Olympics\n\n==Results summary==\n\n===Women's tournament===\n\n====Playoffs====\n\n=====Gold medal game=====\n\n''Sunday, 20 February, 9:05''\n{{#lst:Curling at the 2022 Winter Olympics – Women's tournament|GM}}\n{{Player percentages\n| team1 = {{flagIOC|JPN|2022 Winter}}\n| [[Yurika Yoshida]] | 97%\n| [[Yumi Suzuki]] | 82%\n| [[Chinami Yoshida]] | 64%\n| [[Satsuki Fujisawa]] | 69%\n| teampct1 = 78%\n| team2 = {{flagIOC|GBR|2022 Winter}}\n| [[Hailey Duff]] | 90%\n| [[Jennifer Dodds]] | 89%\n| [[Vicky Wright]] | 89%\n| [[Eve Muirhead]] | 88%\n| teampct2 = 89%\n}}"

relatedness=0.869


'Curling at the 2022 Winter Olympics\n\n==Results summary==\n\n===Mixed doubles tournament===\n\n====Playoffs====\n\n=====Gold medal game=====\n\n\'\'Tuesday, 8 February, 20:05\'\'\n{{#lst:Curling at the 2022 Winter Olympics – Mixed doubles tournament|GM}}\n{| class="wikitable"\n!colspan=4 width=400|Player percentages\n|-\n!colspan=2 width=200 style="white-space:nowrap;"| {{flagIOC|ITA|2022 Winter}}\n!colspan=2 width=200 style="white-space:nowrap;"| {{flagIOC|NOR|2022 Winter}}\n|-\n| [[Stefania Constantini]] || 83%\n| [[Kristin Skaslien]] || 70%\n|-\n| [[Amos Mosaner]] || 90%\n| [[Magnus Nedregotten]] || 69%\n|-\n| \'\'\'Total\'\'\' || 87%\n| \'\'\'Total\'\'\' || 69%\n|}'

relatedness=0.868


"Curling at the 2022 Winter Olympics\n\n==Medal summary==\n\n===Medalists===\n\n{| {{MedalistTable|type=Event|columns=1}}\n|-\n|Men<br/>{{DetailsLink|Curling at the 2022 Winter Olympics – Men's tournament}}\n|{{flagIOC|SWE|2022 Winter}}<br>[[Niklas Edin]]<br>[[Oskar Eriksson]]<br>[[Rasmus Wranå]]<br>[[Christoffer Sundgren]]<br>[[Daniel Magnusson (curler)|Daniel Magnusson]]\n|{{flagIOC|GBR|2022 Winter}}<br>[[Bruce Mouat]]<br>[[Grant Hardie]]<br>[[Bobby Lammie]]<br>[[Hammy McMillan Jr.]]<br>[[Ross Whyte]]\n|{{flagIOC|CAN|2022 Winter}}<br>[[Brad Gushue]]<br>[[Mark Nichols (curler)|Mark Nichols]]<br>[[Brett Gallant]]<br>[[Geoff Walker (curler)|Geoff Walker]]<br>[[Marc Kennedy]]\n|-\n|Women<br/>{{DetailsLink|Curling at the 2022 Winter Olympics – Women's tournament}}\n|{{flagIOC|GBR|2022 Winter}}<br>[[Eve Muirhead]]<br>[[Vicky Wright]]<br>[[Jennifer Dodds]]<br>[[Hailey Duff]]<br>[[Mili Smith]]\n|{{flagIOC|JPN|2022 Winter}}<br>[[Satsuki Fujisawa]]<br>[[Chinami Yoshida]]<br>[[Yumi Suzuki]]<br>

relatedness=0.867


"Curling at the 2022 Winter Olympics\n\n==Results summary==\n\n===Men's tournament===\n\n====Playoffs====\n\n=====Gold medal game=====\n\n''Saturday, 19 February, 14:50''\n{{#lst:Curling at the 2022 Winter Olympics – Men's tournament|GM}}\n{{Player percentages\n| team1 = {{flagIOC|GBR|2022 Winter}}\n| [[Hammy McMillan Jr.]] | 95%\n| [[Bobby Lammie]] | 80%\n| [[Grant Hardie]] | 94%\n| [[Bruce Mouat]] | 89%\n| teampct1 = 90%\n| team2 = {{flagIOC|SWE|2022 Winter}}\n| [[Christoffer Sundgren]] | 99%\n| [[Rasmus Wranå]] | 95%\n| [[Oskar Eriksson]] | 93%\n| [[Niklas Edin]] | 87%\n| teampct2 = 94%\n}}"

## 3. Ask

In [20]:
def num_tokens(
    text: str,
    model: str = GPT_MODEL
) -> int:
    """Return the number of tokens in a string."""

    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""

    strings, relatednesses = strings_ranked_by_relatedness(query, df)
    introduction = "Use the below articles on the 2022 Winter Olympics to answer the subsequent question. If the answer cannot be found in the articles, write 'I could not find an answer.'"
    question = f"\n\nQuestion: {query}"
    message = introduction
    for string in strings:
        next_article = f"\n\nWikipedia article section:\n'''\n{string}\n'''"
        if (
            num_tokens(message + next_article + question, model=model)
            > token_budget
        ):
            break
        else:
            message += next_article
    return message + question

def ask(
    query: str,
    df: pd.DataFrame = df,
    model: str = GPT_MODEL,
    token_budget: int = 4096 - 500,
    print_message: bool = False,
) -> str:
    """Answers a query using GPT and a dataframe of relevant texts and embeddings."""

    message = query_message(query, df, model = model, token_budget = token_budget)
    if print_message:
        print(message)
    messages = [
        {"role": "system", "content": "You answer questions about the 2022 Winter Olympics."},
        {"role": "user", "content": message},
    ]
    response = client.chat.completions.create(
        model = model,
        messages = messages,
        temperature = 0
    )
    response_message = response.choices[0].message.content
    return response_message

In [21]:
ask("Which athletes won the gold medal in curling at the 2022 Winter Olympics?")

'The athletes who won the gold medal in curling at the 2022 Winter Olympics were Stefania Constantini and Amos Mosaner from Italy in the mixed doubles tournament.'

## Cleanup

In [22]:
%%sql
USE openai_demo;
DROP TABLE IF EXISTS winter_olympics_2022;

++
||
++
++

In [23]:
%%sql
DROP DATABASE IF EXISTS openai_demo;

++
||
++
++

## License

Some code sections in this notebook were adapted from [Question answering using embeddings-based search](https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb) from OpenAI.

MIT License

Copyright (c) 2023 OpenAI

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.